<a href="https://colab.research.google.com/github/Samoed/text_classification_hackaton/blob/main/notebooks/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!gdown 1xPUpT6jMCwd-R8cxr6rZhfIJ1Z4r0RMt

Downloading...
From: https://drive.google.com/uc?id=1xPUpT6jMCwd-R8cxr6rZhfIJ1Z4r0RMt
To: /content/train_dataset_dataset.zip
100% 16.6M/16.6M [00:00<00:00, 38.7MB/s]


In [7]:
!unzip -qf train_dataset_dataset.zip

In [8]:
!pip install -q accelerate evaluate -U

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, BertModel
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from transformers import Trainer, TrainingArguments
import numpy as np
from tqdm.autonotebook import tqdm
from torch import nn

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 512

In [16]:
class BaseDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_len):
        self.data = dataframe
        self.text = dataframe['text'].tolist()
        self.targets = None
        if 'class' in dataframe:
            self.targets = dataframe['class'].tolist()
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())

        inputs = self.tokenizer(
            "query: "+text,
            add_special_tokens=True,
            max_length=self.max_seq_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_tensors="pt",
        )
        text = inputs['input_ids'].squeeze(0)
        attention = inputs['attention_mask'].squeeze(0)
        token_type_ids = inputs["token_type_ids"].squeeze(0)


        if self.targets is not None:
            return {"input_ids": text, "attention_mask": attention, "token_type_ids": token_type_ids, 'labels': torch.tensor(self.targets[index], dtype=torch.long)}
        else:
            return {"input_ids": text, "attention_mask": attention, "token_type_ids": token_type_ids}


    def __len__(self) -> int:
        return len(self.text)

In [11]:
train_data = pd.read_csv("dataset/sample.csv")

train_data.head()

,class,text
0,arrangement,СОГЛАШЕНИЕ N 8\nо расторжении трудового догово...
1,arrangement,Соглашение о предоставлении опциона на заключе...
2,arrangement,Соглашение\nо реструктуризации задолженности\n...
3,arrangement,Дополнительное соглашение\r\nк договору купли-...
4,arrangement,Соглашение\nо расторжении договора об оказании...


In [12]:
le = LabelEncoder()

train_data["class"] = le.fit_transform(train_data["class"])
train_data.head()

,class,text
0,2,СОГЛАШЕНИЕ N 8\nо расторжении трудового догово...
1,2,Соглашение о предоставлении опциона на заключе...
2,2,Соглашение\nо реструктуризации задолженности\n...
3,2,Дополнительное соглашение\r\nк договору купли-...
4,2,Соглашение\nо расторжении договора об оказании...


In [14]:
train_split, val_split = train_test_split(train_data, train_size=0.8, random_state=42, stratify=train_data["class"])

In [15]:
def read_model(model_name: str = "DeepPavlov/rubert-base-cased"):
    model = BertForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(le.classes_))
#     model = BertModel.from_pretrained(model_name, return_dict=True)
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    return model, tokenizer

In [17]:
model, tokenizer = read_model("intfloat/multilingual-e5-small") # intfloat/multilingual-e5-large
# model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMRobertaTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [18]:
train_dataset = BaseDataset(train_split, tokenizer, MAX_LEN)
val_dataset = BaseDataset(val_split, tokenizer, MAX_LEN)

In [19]:
import evaluate
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [20]:
from transformers import SchedulerType

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,
    evaluation_strategy="epoch",     # perform evaluation each epoch
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=64,
    auto_find_batch_size=True,
    save_strategy="epoch",
    report_to=None,
    lr_scheduler_type=SchedulerType.COSINE_WITH_RESTARTS,
    logging_first_step=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,2.267800,2.052189,0.518428
2,1.926600,1.743640,0.794599
3,1.692600,1.588083,0.841897
4,1.610200,1.559144,0.874815


TrainOutput(global_step=200, training_loss=1.874911117553711, metrics={'train_runtime': 128.1899, 'train_samples_per_second': 12.481, 'train_steps_per_second': 1.56, 'total_flos': 105413399347200.0, 'train_loss': 1.874911117553711, 'epoch': 4.0})